### Grafovske neuronske mreže

Pravilo propagacije kod grafovskih neuronskih mreža je definisano na sledeći način:

$$
H^{(k+1)} = \sigma \bigl(\hat{A}H^{(k)}W^{(k)}\bigr)
$$

gde je sa $k$ numerisan sloj mreže, $\sigma$ je aktivaciona funkcija, $H^{(0)}=X$, tj. ulazna matrica prediktora, $\hat{A} = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}$ simetrična normalizovana matrica susedstva grafa $G$ sa dodatim petljama za svaki čvor, gde je $\tilde{A} = A+I_N$ ($N$ je broj čvorova, $A$ je matrica susedstva) i $\tilde{D}_{ii} = \sum_{j}^{}\tilde{A}_{ij}$, $W^{(k)}$ je matrica parametara modela za sloj $k$.
Matrica susedstva se normalizuje da bi proces optimizacije kroz duboku neuronsku mrežu bio stabilniji.
Dakle, arhitektura je slična kao kod potpuno povezanih neuronskih mreža, samo što je dodata i matrica susedstva čvorova grafa.
<br/> Primer grafovske neuronske mreže sa dva sloja je:

$$
f(X,A) = softmax\bigl(\hat{A} \hspace{1.2mm} LeakyReLU(\hat{A}XW^{(0)})W^{(1)}\bigr)
$$

Gde je $W^{(0)} \in \mathbb{R}^{d \times h^{(0)}}$ matrica parametara za sloj koji spaja ulazne podatke dimenzije $d$ i podatke prvog skrivenog sloja dimenzije $h^{(0)}$, a $W^{(1)} \in \mathbb{R}^{h^{(0)} \times h^{(1)}}$ je matrica parametara za sloj koji spaja skrivene podatke i izlazne podatke.
Na izlazu iz ulaznog sloja primenjuje se aktivaciona funkcija LeakyReLU, a na izlazu iz skrivenog sloja primenjuje se aktivaciona funkcija softmax. Ove dve funkcije su definisane sledećim formulama:

$$
LeakyReLU(x) = max(0.1x, x)
$$

$$
softmax(x_j) = \frac{exp(x_j)}{\sum_{i}^{}exp(x_i)}
$$

### Grafovske konvolutivne mreže (eng. *graph convolutional network*)

### Mehanizam pažnje nad grafom

Ulazni podaci za sloj sa mehanizmom pažnje čine skup $h=\{h_1,h_2,...,h_N\}$, $h_i \in \mathbb{R}^F$, gde je $N$ broj čvorova grafa, a $F$ je dimenzija atributa za svaki čvor. Izlazni podaci ovog sloja čine novi skup atributa $\{h_1',h_2',...,h_N'\}$, $h_i' \in \mathbb{R}^{F'}$, potencijalno različite dimenzije $F'$.
Izlazni podaci se računaju sledećom formulom:

$$
h_i' = \sigma\biggl(\sum_{j \in N_i}^{} \alpha_{ij}Wh_j\biggr)
$$

gde je $\sigma$ aktivaciona funkcija, $N_i$ skup suseda $i$-tog čvora, $W \in \mathbb{R}^{F'\times F} $ matrica parametara koja je ista za svaki čvor, $\alpha_{ij}$ normalizovani koeficijenti pažnje (eng. *normalised attention coefficients*).
Normalizovani koeficijent pažnje $\alpha_{ij}$ nam govori o važnosti atributa $j$-tog čvora za $i$-ti čvor. Postupak za njegovo izračunavanje dat je sledećom formulom:

$$
\alpha_{ij} = \frac{exp(e_{ij})}{\sum_{k \in N_i}^{}exp(e_{ik})}
$$

gde su $e_{ik}$ koeficijenti pažnje i oni se računaju sledećom formulom:

$$
e_{ij} = a(Wh_i, Wh_j)
$$

gde je $a: \mathbb{R}^{F'} \times \mathbb{R}^{F'} \rightarrow \mathbb{R}$ funkcija koja se naziva mehanizmom pažnje (eng. *attention mechanism*).
U našem zadatku će mehanizam pažnje biti jednoslojna neuronska mreža parametrizovana vektorom $\vec{a} \in \mathbb{R}^{2F'}$, na koju se primenjuje aktivaciona funkcija LeakyReLU definisana sa:

$$
LeakyReLU(x) = max(0.2x, x)
$$
Dakle, formula za izračunavanje normalizovanih koeficijenata pažnje je data sa:

$$
\alpha_{ij} = \frac{exp\bigl(LeakyReLU\bigl(\vec{a}^T[Wh_i\parallel Wh_j]\bigr)\bigr)}{\sum_{k \in N_i}^{}exp\bigl(LeakyReLU\bigl(\vec{a}^T[Wh_i\parallel Wh_j]\bigr)\bigr)}
$$

gde je $\parallel$ operacija nadovezivanja, a $^{T}$ operacija transponovanja.

Radi stabilizacije procesa učenja pažnje, koristi se mehanizam višestruke pažnje (eng. *multi-head attention*), u kojem koristimo $K$ nezavisnih mehanizama pažnje i pomoću svakog dobijamo izlazne atribute. Nakon toga se izlazni atributi nadovezuju i dobijamo sledeću reprezentaciju izlaznih atributa:

$$
h_i' = \parallel\sigma\biggl(\sum_{j \in N_i}^{}\alpha_{ij}^kW_kh_j\biggr)
$$

gde je $\parallel$ operacija nadovezivanja atributa za $k$ od 1 do $K$, $\alpha_{ij}^k$ su normalizovani koeficijenti pažnje izračunati pomoću $k$-tog mehanizma pažnje, $W_k$ je odgovarajuća matrica parametara. Primetimo da će u ovom slučaju dimenzija izlaznog čvora biti $KF'$. Posebno, u poslednjem sloju neuronske mreže umesto nadovezivanja vrši se uprosečavanje, kako bi se dobio izlaz odgovarajuće dimenzije, nakon čega se vrši konačna nelinearna transformacija (npr. softmax u slučaju klasifikacije). Dakle, izlazni atributi poslednjeg sloja su dati formulom:

$$
h_i' = \sigma\biggl(\frac{1}{K}\sum_{k=1}^{K}\sum_{j \in N_i}^{}\alpha_{ij}^kW_kh_j\biggr)
$$